In [1]:
# Import all the needed libraries
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import algos
import os
if not pt.started():
    pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
# Load the dataset
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

In [3]:
index_ref = pt.IndexRef.of(os.getcwd() + "./passage_index/data.properties")

In [4]:
index = pt.IndexFactory.of(index_ref)

18:33:42.541 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.


In [5]:
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [6]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [7]:
qrels = dataset.get_qrels("test-2019")
res = algos.get_all_scores(dataset, bm25)

18:33:42.686 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [8]:
pt.Experiment(
  [bm25],
  dataset.get_topics("test-2019"), 
  dataset.get_qrels("test-2019"),
  eval_metrics=["ndcg", "map", "recip_rank"], 
  # perquery = "true",
  # baseline=0,
  round=3,
)

18:34:23.118 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


,name,ndcg,map,recip_rank
0,BR(BM25),0.593,0.37,0.795


In [9]:
eval = pt.Utils.evaluate(res, qrels, metrics = ["recip_rank"], perquery = True)

In [ ]:
# only 43 queries with qrels available in the collection

In [ ]:
eval

In [ ]:
# we need for each query id the most relevant document, and the recip procal rank and the query itself
# [[qid], [RR], [query]]

In [ ]:
qids = [i for i in eval]
RR = [eval[j]['recip_rank'] for j in eval]

info = []
    
for qid in qids:
    info.append(str(res[res['qid']==str(qid)].iloc[0]["query"]))
                
print(info)

In [ ]:
queries = pd.DataFrame({"qid":qids,"mrr":RR,"query":info})


In [ ]:
queries = queries[21:]

In [ ]:
queries.to_csv("Query20Part2.csv",index=False)

In [ ]:
# Return the top-1 ranked document for all queries in the test set
def get_topRanked_doc(queries, index_ref):
    docs = []
    # count = 0
    for query in list(queries['query']):
        # if count == 20 : break
        # Search for the passage given the docno
        print(query)
        df = pt.BatchRetrieve(index_ref).search(query)[0:1]
        doc = (algos.search_with_doc_id(int(df['docid'])))
        docs.append(len(doc.split()))
        # count += 1
    return docs

In [ ]:
docs_lengths = get_topRanked_doc(queries, index_ref)
query_lengths = [len(query.split()) for query in list(queries['query'])]

In [ ]:
queries['doc_length'] = docs_lengths
queries['query_length'] = query_lengths

In [ ]:
queries

In [ ]:
queries.to_csv("Query20Part2WithLengths.csv",index=False)

In [ ]:
plt.scatter(x = queries[queries["mrr"]>0.5]["doc_length"], y = queries[queries["mrr"]>0.5]["query_length"],
           marker='D',
           color='red',
           label='score > 0.5')
plt.scatter(x =  queries[queries["mrr"]<=0.5]["doc_length"], y = queries[queries["mrr"]<=0.5]["query_length"] ,
           marker='o',
           color='blue',
           label='score <= 0.5')
plt.title('Query length vs. Document length')
plt.xlabel('Document length')
plt.ylabel('Query length')
plt.legend(title="MRR", loc=4, fontsize='small', fancybox=True)
plt.show()

In [ ]:
# We assumed that longer query, we also assumed that the MRR would be higher but that does not seem the case

In [ ]:
df1 = pd.read_csv("Query20WithLengths.csv")

In [ ]:
frames = [df1, queries]
result = pd.concat(frames)
result

In [ ]:
plt.scatter(x = result[result["mrr"]>0.5]["doc_length"], y = result[result["mrr"]>0.5]["query_length"],
           marker='D',
           color='red',
           label='score > 0.5')
plt.scatter(x =  result[result["mrr"]<=0.5]["doc_length"], y = result[result["mrr"]<=0.5]["query_length"] ,
           marker='o',
           color='blue',
           label='score <= 0.5')
plt.title('Query length vs. Document length')
plt.xlabel('Document length')
plt.ylabel('Query length')
plt.legend(title="MRR", loc=4, fontsize='small', fancybox=True)
plt.savefig('QueryLengthVSDocumentLength.svg')

In [ ]:
result.to_csv("QueriesAllLengths.csv",index=False)

In [ ]:
result = pd.read_csv("QueriesAllLengths.csv")

In [ ]:
score = []
for qid in list(result["qid"]):
    df1 = res[(res["qid"]==str(qid)) & (res["rank"]==0)]
    score.append(float(df1["score"]))
    #info.append(str(res[res['qid']==str(qid)].iloc[0]["query"]))
result["score"] = score

In [ ]:
result.to_csv("QueriesAllLengthScore.csv", index=False)

In [ ]:
plt.scatter(result['doc_length'], result['score'])
plt.title('Relevancy vs. Document length')
plt.xlabel('Document length')
plt.ylabel('Relevancy')
# plt.legend(title="MRR", loc=4, fontsize='small', fancybox=True)
plt.savefig('DocumentVsRelvancy.svg')
plt.show()

In [ ]:
import numpy as np
import statistics

list1 = result[result['query_length']==1]['score']
print("list1:", list1)
list2 = result[result['query_length']==2]['score']
print("\nlist2:",list2)
list3 = result[result['query_length']==3]['score']
list4 = result[result['query_length']==4]['score']
list5 = result[result['query_length']==5]['score']
list6 = result[result['query_length']==6]['score']
list7 = result[result['query_length']==7]['score']
list8 = result[result['query_length']==8]['score']
list9 = result[result['query_length']==9]['score']

print("Max query len = ",max(result['query_length']))
plt.boxplot([list1,list2,list3,list4,list5,list6,list7,list8,list9])

plt.plot(range(2,10),
         [statistics.median(list2),statistics.median(list3),statistics.median(list4),statistics.median(list5),statistics.median(list6),statistics.median(list7),statistics.median(list8),statistics.median(list9)])

plt.title('Relevancy vs. Query length')
plt.xlabel('Query length')
plt.ylabel('Relevancy')
# plt.legend(title="MRR", loc=4, fontsize='small', fancybox=True)
plt.savefig('QueryVsRelvancy.svg')
plt.show()

In [ ]:
####################################################################################################################
#####WHAT IS THE RESON THAT QUERIES FOR WHICH THE QREL IS PROVIDED, NO DOCUMENTS ARE RETRIEVED IN THE TOP1000?######
####################################################################################################################

In [117]:
import gc
#def given qrels dataset, return if qid, docid exists in ranked model
def check_qrel_queries():
    temp1 = dataset.get_topicsqrels("train")[0]
    queries_df = temp1.sort_values(by=['qid']).head(1000)
    temp2 = dataset.get_topicsqrels("train")[1]
    qrels_df = temp2.sort_values(by=['qid']).head(1000)
    
    docs_qrels = pd.DataFrame(columns =['rank','docid','score','qid'])

    for qid in list(queries_df["qid"]):
        # One particular query
        qrels_queries = qrels_df[qrels_df["qid"] == qid]
        
        if len(qrels_queries) == 0:
            # print("qid:" + str(qid) + "does not exist")
            continue
            
        docNo = qrels_queries.iloc[0]["docno"]        
        query = queries_df[queries_df["qid"] == qid].iloc[0]["query"]
        
        # Returns the top 1000 relevant documents for this query (that has as qrel)
        result = bm25.search(query)
        
        docs_qrels = docs_qrels.append(check_rel_doc_present(docNo, qid, query, result), ignore_index=True)
        
        del result
        del qrels_queries
        gc.collect()
        
    
    return docs_qrels

#given a docid, search if the doc id exists in result.
def check_rel_doc_present(docid, qid, query, result):

    # Hypothesis that this if statement is always true
    if docid in list(result["docno"]):
        R = result[result["docno"]==docid]
        rank = R.iloc[0]["rank"]
        score = R.iloc[0]["score"]
        doc_id = R.iloc[0]["docid"]
        # print("rank : ", rank)
        # print("score: ", score)
        # print("docid : ", doc_id)
        return {"rank":rank, "docid":doc_id, "score":round(score,2), "qid":qid, "query":query}
    else:
        # Then we have document that were not found in the top-1000 ranked documents despite 
        # the query they belong to had a qrel provided
        
        # print("not found")
        return {"rank":-1, "docid":docid, "score":-1, "qid":qid, "query":query}

In [118]:
temp = check_qrel_queries()

19:25:27.374 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8
19:25:42.673 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [119]:
temp

,rank,docid,score,qid,query
0,2,3837260,41.86,100,of earths crust is dysprosium
1,3,1936056,55.86,10000,about what proportion of quebec s population s...
2,190,3617179,17.54,1000005,where is westminster california
3,38,5939245,20.03,1000007,where is westminster ma
4,0,4630854,31.57,1000008,where is westminster md
...,...,...,...,...,...
686,998,2011677,16.78,100125,cortana who are the forerunners
687,94,5714439,21.75,100126,cortana who is arbiter
688,-1,4670229,-1.00,1001260,where to buy make up from accessorize uk
689,2,4402712,68.32,1001262,where to buy maui jim prescription sunglasses


In [121]:
temp.to_csv("QUERIES_W_QRESL_RETRIEVED_AND_NOT_RETRIEVED.csv", index=False)

In [112]:
# Now we are going to sort the queries bases on their rank aka if rank == -1 we are putting those in a new dataframe

In [113]:
DOCS = pd.read_csv("./collection.tsv", sep='\t', header=1, names=["passage"])


In [114]:
DOCS

,passage
2,Essay on The Manhattan Project - The Manhattan...
3,The Manhattan Project was the name for a proje...
4,versions of each volume as well as complementa...
5,The Manhattan Project. This once classified ph...
6,Nor will it attempt to substitute for the extr...
...,...
8841818,When metal salts emit short wavelengths of vis...
8841819,Thousands of people across the United States w...
8841820,"The recipe that creates blue, for example, inc..."
8841821,"On Independence Days of yore, old-timey crowds..."


In [122]:
queries_without_doc_retrieved = temp[temp['score']==(-1.00)]
# THESE ARE THE QUERIES FOR WHICH A QREL IS PROVIDED, BUT THE MODEL DID NOT RETRIEVE A DOCUMENT (WAS NOT IN THE 
# TOP 1000)

In [123]:
queries_without_doc_retrieved = queries_without_doc_retrieved.head(10)

In [124]:
queries_without_doc_retrieved

,rank,docid,score,qid,query
29,-1,2640014,-1.0,1000041,where is wheat grown mainly state
36,-1,4496957,-1.0,1000052,where is whip coral harvested
53,-1,4660814,-1.0,1000089,where is whitehall kentucky
60,-1,3943327,-1.0,100010,cortana what is polygamy
94,-1,3035531,-1.0,100015,cortana what is the average blood pressure
128,-1,2441538,-1.0,1000204,where is wilson co tn county seat
130,-1,3797484,-1.0,1000207,where is wilson nc
140,-1,2964624,-1.0,100022,cortana what is the definition of love
143,-1,4393147,-1.0,1000226,where is windows program located
151,-1,2372474,-1.0,100024,cortana what is the do carrots


In [125]:
# RETRIEVE THESE DOCUMENTS TO SEE WHAT IS WRONG
passages = [(DOCS.iloc[(int(x) - 1)]['passage']) for x in queries_without_doc_retrieved['docid']]

In [126]:
queries_without_doc_retrieved['passages'] = passages

In [127]:
# FOR THOSE QUERIES WITH A QREL, NOW THE PASSAGES ARE FOUND BY US IN THE DOCUMENT AND ATTACHED
queries_without_doc_retrieved

,rank,docid,score,qid,query,passages
29,-1,2640014,-1.0,1000041,where is wheat grown mainly state,"Punjab, Haryana, Uttar Pradesh, Rajasthan and ..."
36,-1,4496957,-1.0,1000052,where is whip coral harvested,"Clearly, this is an important field of study f..."
53,-1,4660814,-1.0,1000089,where is whitehall kentucky,Jupiter's moons. Each of the gas giant planets...
60,-1,3943327,-1.0,100010,cortana what is polygamy,Recent Examples of polygamy from the Web. 1 B...
94,-1,3035531,-1.0,100015,cortana what is the average blood pressure,"Ideally, we should all have a blood pressure b..."
128,-1,2441538,-1.0,1000204,where is wilson co tn county seat,Written by: Amelia Morrison Hipps September 10...
130,-1,3797484,-1.0,1000207,where is wilson nc,There could not be a better start for my unive...
140,-1,2964624,-1.0,100022,cortana what is the definition of love,Examples of jones in a sentence. 1 that dude ...
143,-1,4393147,-1.0,1000226,where is windows program located,"To open the certain program folder, -path pres..."
151,-1,2372474,-1.0,100024,cortana what is the do carrots,Not all of the chemical constituents of carrot...


In [ ]:
queries_without_doc_retrieved[queries_without_doc_retrieved['qid'] == str(1000041)]['passages']

In [128]:
queries_without_doc_retrieved.to_csv("QUERIES_WITHOUT_DOC_IN_RETRIEVAL_WITHQREL.csv", index=False)

In [ ]:
# Model built with all the queries (regardless of they have qrels), and evaluation is where we are using 
# the qrels to determine the relevancy (intstrument to give the final score)

# Given the query, with its qrel, these documents were not retrieved.
# So either the query is wrong or the less likely case is that the qrel is wrong
# Assuming the query is wrong, possibilities that causes the system to malfunctio the system is because...
# - Perhaps the document is not classified as relevant because either the query or the passage using synomyms 

In [ ]:
# ERROR ANALYSIS - BM25 - ALL
# - (WIP)Julian: Need to determine for which queries (with qrel) the document is not retrived?
#   - And provided reasons for this
# - (WIP/DONE)Julian:Least performing queries (with qrels), provide the best performing documents,
#    what can we say about them?
# - (see Google docs for extra is time allows)
# IMPROVEMENTS - BM25 (ask the teacher if we need to do error analysis on these as well? - nice to have)
# - Julian : Query expansion 
# - Varnika + Monice : error analysis
# - All : SIGNIFICANCE TESTING BETWEEN BM25 AND IMPROVEMENTS (see Pyterrier)
#---------------------------------------------------------------------------------------------------------------------
# Julian : INDEXING L2R
# - WIP
# Julian : IMPLEMENTING BASELINE (L2R)
# - WIP
# (maybe verify with the teacher just to be sure that this is everything that needs to be done)
#---------------------------------------------------------------------------------------------------------------------
# REPORT (All)
# - Update the report according to the feedback we receive from the teacher/TA
# - Extend the report where needed
#---------------------------------------------------------------------------------------------------------------------
# STUDY!
# - Queations about the prodcut, reason why we did something instead of something else
# - Or ask questions about theory of the course, and why we choose to use BM25 instead of other model
#---------------------------------------------------------------------------------------------------------------------
# (DONE)Julian : send email Claudi + GitHub
# - Should do the error analysis also on the improvements similar to what we had for the baseline BM25?
# - Do the images count towards the page limit?
#---------------------------------------------------------------------------------------------------------------------
# Next meeting either Thursday or Friday